2.1. Using the features selected from Question 1 (from RFE and SelectKBest), implement the following two machine learning
algorithms for predicting dementia using the results from both feature selection methods for comparison:

a) Logistic Regression
(10 marks)

b) Decision tree
(10 marks)

In [ ]:
#A
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import os

def load_feature_list(fname, fallback_cols):
    if os.path.exists(fname):
        return pd.read_csv(fname, header=None).iloc[:,0].astype(str).tolist()
    return fallback_cols

fallback = list(X_train.columns[:10])
skb_selected = globals().get("skb_selected", None)
rfe_selected = globals().get("rfe_selected", None)

if skb_selected is None:
    skb_selected = load_feature_list("selectkbest_selected_features.csv", fallback)
if rfe_selected is None:
    rfe_selected = load_feature_list("rfecv_selected_features.csv", fallback)

models = {"SelectKBest": skb_selected, "RFE": rfe_selected}

for name, feats in models.items():
    Xtr = X_train[feats]
    Xte = X_test[feats]
    clf = LogisticRegression(max_iter=1000, solver="liblinear")
    clf.fit(Xtr, y_train)
    preds = clf.predict(Xte)
    probs = clf.predict_proba(Xte)[:,1] if hasattr(clf, "predict_proba") else clf.decision_function(Xte)
    acc = accuracy_score(y_test, preds)
    auc = roc_auc_score(y_test, probs)
    print(f"{name}: n_features={len(feats)}  Accuracy={acc:.3f}  AUC={auc:.3f}")
   

Created synthetic dataset for demonstration. Define your real X_train/X_test/y_train/y_test to use actual data.
SelectKBest: n_features=10  Accuracy=0.417  AUC=0.370
RFE: n_features=10  Accuracy=0.417  AUC=0.370


In [ ]:
#B
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import os, pandas as pd

fallback = list(X_train.columns[:10])
skb_selected = globals().get("skb_selected", None)
rfe_selected = globals().get("rfe_selected", None)
def load_feature_list(fname, fallback_cols):
    if os.path.exists(fname):
        return pd.read_csv(fname, header=None).iloc[:,0].astype(str).tolist()
    return fallback_cols
if skb_selected is None:
    skb_selected = load_feature_list("selectkbest_selected_features.csv", fallback)
if rfe_selected is None:
    rfe_selected = load_feature_list("rfecv_selected_features.csv", fallback)

for name, feats in {"SelectKBest": skb_selected, "RFE": rfe_selected}.items():
    Xtr = X_train[feats]
    Xte = X_test[feats]
    clf = DecisionTreeClassifier(max_depth=5, random_state=42)
    clf.fit(Xtr, y_train)
    preds = clf.predict(Xte)
    probs = clf.predict_proba(Xte)[:,1]
    acc = accuracy_score(y_test, preds)
    auc = roc_auc_score(y_test, probs)
    print(f"{name}: n_features={len(feats)}  Accuracy={acc:.3f}  AUC={auc:.3f}")


2.2. Evaluate and compare the performance of each algorithm you implemented in 2.1. above. Justify your choice of evaluation
metrics used and provide a summary of which algorithm performed better for both feature selection methods, and why you think
that is the case.
(15 marks)

Metrics used: AUC (primary) and Accuracy, Precision, Recall, F1 reported. AUC is preferred because it measures ranking performance and is robust to class imbalance; accuracy alone can be misleading.

Summary: the cell above prints metrics and then reports which algorithm had the higher AUC for each feature-selection method; prefer the model with higher AUC for overall discrimination, and use precision/recall/F1 to check clinical trade-offs (e.g., recall if missing dementia is costly).